In [1]:
# Работа с признаками. Практика

import os

import pandas as pd
import mlflow
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import (
    OneHotEncoder, 
    SplineTransformer, 
    QuantileTransformer, 
    RobustScaler,
    PolynomialFeatures,
    KBinsDiscretizer,
)
from dotenv import load_dotenv

TABLE_NAME = "users_churn" # таблица с данными

TRACKING_SERVER_HOST = "127.0.0.1"
TRACKING_SERVER_PORT = 5000

EXPERIMENT_NAME = 'churn_kruglikovAlex' # название эксперимента
RUN_NAME = "preprocessing" 
REGISTRY_MODEL_NAME = 'churn_model_kruglikovAlex_b2c' # название зарегистрированной модели 

In [2]:
pd.options.display.max_columns = 100
pd.options.display.max_rows = 64

In [3]:
# подгружаем .env
load_dotenv()

# загрузка данных для обучения модели
connection = {"sslmode": "require", "target_session_attrs": "read-write"}
postgres_credentials = {
    "host": os.getenv("DB_DESTINATION_HOST"),
    "port": os.getenv("DB_DESTINATION_PORT"),
    "dbname": os.getenv("DB_DESTINATION_NAME"),
    "user": os.getenv("DB_DESTINATION_USER"),
    "password": os.getenv("DB_DESTINATION_PASSWORD"),
}

In [6]:
pip install psycopg2-binary

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [ ]:
connection.update(postgres_credentials)

import psycopg

with psycopg.connect(**connection) as conn:

    with conn.cursor() as cur:
        cur.execute(f"SELECT * FROM {TABLE_NAME}")
        data = cur.fetchall()
        columns = [col[0] for col in cur.description]

df = pd.DataFrame(data, columns=columns)

In [7]:
df.head(5)

,id,customer_id,begin_date,end_date,type,paperless_billing,payment_method,monthly_charges,total_charges,internet_service,online_security,online_backup,device_protection,tech_support,streaming_tv,streaming_movies,gender,senior_citizen,partner,dependents,multiple_lines,target
0,28173,7590-VHVEG,2020-01-01,NaT,Month-to-month,Yes,Electronic check,29.85,29.85,DSL,No,Yes,No,No,No,No,Female,0,Yes,No,None,0
1,28174,5575-GNVDE,2017-04-01,NaT,One year,No,Mailed check,56.95,1889.50,DSL,Yes,No,Yes,No,No,No,Male,0,No,No,No,0
2,28175,3668-QPYBK,2019-10-01,2019-12-01,Month-to-month,Yes,Mailed check,53.85,108.15,DSL,Yes,Yes,No,No,No,No,Male,0,No,No,No,1
3,28176,7795-CFOCW,2016-05-01,NaT,One year,No,Bank transfer (automatic),42.30,1840.75,DSL,Yes,No,Yes,Yes,No,No,Male,0,No,No,None,0
4,28177,9237-HQITU,2019-09-01,2019-11-01,Month-to-month,Yes,Electronic check,70.70,151.65,Fiber optic,No,No,No,No,No,No,Female,0,No,No,No,1


In [10]:
obj_df = df.select_dtypes(include="object")
obj_df[:5]

,customer_id,type,paperless_billing,payment_method,internet_service,online_security,online_backup,device_protection,tech_support,streaming_tv,streaming_movies,gender,partner,dependents,multiple_lines
0,7590-VHVEG,Month-to-month,Yes,Electronic check,DSL,No,Yes,No,No,No,No,Female,Yes,No,None
1,5575-GNVDE,One year,No,Mailed check,DSL,Yes,No,Yes,No,No,No,Male,No,No,No
2,3668-QPYBK,Month-to-month,Yes,Mailed check,DSL,Yes,Yes,No,No,No,No,Male,No,No,No
3,7795-CFOCW,One year,No,Bank transfer (automatic),DSL,Yes,No,Yes,Yes,No,No,Male,No,No,None
4,9237-HQITU,Month-to-month,Yes,Electronic check,Fiber optic,No,No,No,No,No,No,Female,No,No,No


In [11]:
# Задание 1
# Преобразуйте категориальные колонки с помощью OneHotEncoding. Используйте автоопределение 
# категорий. Поставьте ограничение на максимальное количество категорий — 10, игнорируйте неизвестные 
# категории во время преобразования и удалите первую категорию, чтобы исключить проблему коллинеарности.

# определение категориальных колонок, которые будут преобразованы
cat_columns = ["type", "payment_method", "internet_service", "gender"]

In [46]:
# создание объекта OneHotEncoder для преобразования категориальных переменных
# auto - автоматическое определение категорий
# ignore - игнорировать ошибки, если встречается неизвестная категория
# max_categories - максимальное количество уникальных категорий
# sparse_output - вывод в виде разреженной матрицы, если False, то в виде обычного массива
# drop="first" - удаляет первую категорию, чтобы избежать ловушки мультиколлинеарности
encoder_oh = OneHotEncoder(categories='auto', handle_unknown='ignore', max_categories=10, sparse_output=False, drop='first')


In [47]:
encoder_oh

OneHotEncoder(drop='first', handle_unknown='ignore', max_categories=10,
              sparse_output=False)

In [59]:
encoded_features = encoder_oh.fit_transform(df[cat_columns].to_numpy())

In [60]:
encoded_features

array([[0., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 0., 1.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 1., 0., 1.],
       [0., 1., 0., ..., 1., 0., 1.]])

In [61]:
# преобразование полученных признаков в DataFrame и установка названий колонок
# get_feature_names_out() - получение имён признаков после преобразования
encoded_df = pd.DataFrame(encoded_features, columns=encoder_oh.get_feature_names_out())
encoder_oh.get_feature_names_out()

array(['x0_One year', 'x0_Two year', 'x1_Credit card (automatic)',
       'x1_Electronic check', 'x1_Mailed check', 'x2_Fiber optic',
       'x2_None', 'x3_Male'], dtype=object)

In [62]:
encoded_df.head()

,x0_One year,x0_Two year,x1_Credit card (automatic),x1_Electronic check,x1_Mailed check,x2_Fiber optic,x2_None,x3_Male
0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
1,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0
2,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0
3,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0


In [63]:
encoder_oh.get_feature_names_out().reshape(1,-1)

array([['x0_One year', 'x0_Two year', 'x1_Credit card (automatic)',
        'x1_Electronic check', 'x1_Mailed check', 'x2_Fiber optic',
        'x2_None', 'x3_Male']], dtype=object)

In [64]:
pd.DataFrame(encoded_df)

,x0_One year,x0_Two year,x1_Credit card (automatic),x1_Electronic check,x1_Mailed check,x2_Fiber optic,x2_None,x3_Male
0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
1,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0
2,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0
3,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...
7038,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0
7039,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
7040,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
7041,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0


In [65]:
# конкатенация исходного DataFrame с новым DataFrame, содержащим закодированные категориальные признаки
# axis=1 означает конкатенацию по колонкам
obj_df = pd.concat([obj_df, encoded_df], axis=1)

obj_df.head(2)

,customer_id,type,paperless_billing,payment_method,internet_service,online_security,online_backup,device_protection,tech_support,streaming_tv,streaming_movies,gender,partner,dependents,multiple_lines,type_One year,type_Two year,payment_method_Credit card (automatic),payment_method_Electronic check,payment_method_Mailed check,internet_service_Fiber optic,internet_service_None,gender_Male,x0_One year,x0_Two year,x1_Credit card (automatic),x1_Electronic check,x1_Mailed check,x2_Fiber optic,x2_None,x3_Male
0,7590-VHVEG,Month-to-month,Yes,Electronic check,DSL,No,Yes,No,No,No,No,Female,Yes,No,None,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
1,5575-GNVDE,One year,No,Mailed check,DSL,Yes,No,Yes,No,No,No,Male,No,No,No,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0


In [66]:
# Задание 2
# Напишите код преобразования числовых признаков в списке num_columns, используя следующие энкодеры:
# SplineTransformer,
# QuantileTransformer,
# RobustScaler,
# PolynomialFeatures,
# KBinsDiscretizer.

In [101]:
data = pd.read_csv('/home/mle-user/mle_projects/mle-dvc/data/initial_data.csv')

In [102]:
data.head()
df = data

In [103]:
num_columns = ["monthly_charges", "total_charges"]

In [104]:
# параметры преобразователей
n_knots = 3
degree_spline = 4
n_quantiles=100
degree = 3
n_bins = 5
encode = 'ordinal'
strategy = 'uniform'
subsample = None

In [105]:
# SplineTransformer
encoder_spl = SplineTransformer(n_knots=n_knots, degree=degree) # ваш код здесь #
encoder_spl

SplineTransformer(n_knots=3)

In [106]:
encoded_features = encoder_spl.fit_transform(df[num_columns]) # ваш код здесь #

In [107]:
encoded_df = pd.DataFrame(
    encoded_features, 
    columns=encoder_spl.get_feature_names_out(num_columns)
)
num_df = pd.concat([df, encoded_df], axis=1)

In [108]:
num_df.head()

,id,begin_date,end_date,type,paperless_billing,payment_method,monthly_charges,total_charges,internet_service,online_security,online_backup,device_protection,tech_support,streaming_tv,streaming_movies,gender,senior_citizen,partner,dependents,multiple_lines,target,monthly_charges_sp_0,monthly_charges_sp_1,monthly_charges_sp_2,monthly_charges_sp_3,monthly_charges_sp_4,total_charges_sp_0,total_charges_sp_1,total_charges_sp_2,total_charges_sp_3,total_charges_sp_4
0,7020,2020-01-01,NaN,Month-to-month,Yes,Electronic check,29.85,29.85,DSL,No,Yes,No,No,No,No,Female,0,Yes,No,No,0,0.075838,0.619528,0.302584,0.002050,0.000000,0.165395,0.666660,0.167945,2.764200e-09,0.0
1,7021,2017-04-01,NaN,One year,No,Mailed check,56.95,1889.50,DSL,Yes,No,Yes,No,No,No,Male,0,No,No,No,0,0.002024,0.301936,0.619907,0.076133,0.000000,0.030585,0.520509,0.435494,1.341205e-02,0.0
2,7022,2019-10-01,2019-12-01,Month-to-month,Yes,Mailed check,53.85,108.15,DSL,Yes,Yes,No,No,No,No,Male,0,No,No,No,1,0.004130,0.342546,0.594060,0.059264,0.000000,0.156567,0.666246,0.177185,1.461391e-06,0.0
3,7023,2016-05-01,NaN,One year,No,Bank transfer (automatic),42.30,1840.75,DSL,Yes,No,Yes,Yes,No,No,Male,0,No,No,No,0,0.023624,0.492418,0.465686,0.018272,0.000000,0.032438,0.527033,0.428139,1.239059e-02,0.0
4,7024,2019-09-01,2019-11-01,Month-to-month,Yes,Electronic check,70.70,151.65,Fiber optic,No,No,No,No,No,No,Female,0,No,No,No,1,0.000000,0.145721,0.664792,0.189474,0.000014,0.151802,0.665741,0.182452,4.803606e-06,0.0


In [109]:
# QuantileTransformer
encoder_q = QuantileTransformer(n_quantiles=n_quantiles) # ваш код здесь #
encoder_q

QuantileTransformer(n_quantiles=100)

In [110]:
encoded_features = encoder_q.fit_transform(df[num_columns]) # ваш код здесь #
encoded_features


array([[0.23213724, 0.02483202],
       [0.38989899, 0.5789275 ],
       [0.35353535, 0.11649739],
       ...,
       [0.2314934 , 0.22884664],
       [0.5390525 , 0.21302932],
       [0.93497475, 0.94618654]])

In [111]:
encoded_df = pd.DataFrame(
    encoded_features, 
    columns=encoder_q.get_feature_names_out(num_columns)
)# ваш код здесь #
encoded_df.head()

,monthly_charges,total_charges
0,0.232137,0.024832
1,0.389899,0.578927
2,0.353535,0.116497
3,0.267536,0.571996
4,0.505612,0.138627


In [112]:
encoded_df.columns = [col + f"_q_{n_quantiles}" for col in num_columns]
encoded_df.columns

Index(['monthly_charges_q_100', 'total_charges_q_100'], dtype='object')

In [113]:
num_df = pd.concat([num_df, encoded_df], axis=1)

In [114]:
num_df.head()

,id,begin_date,end_date,type,paperless_billing,payment_method,monthly_charges,total_charges,internet_service,online_security,online_backup,device_protection,tech_support,streaming_tv,streaming_movies,gender,senior_citizen,partner,dependents,multiple_lines,target,monthly_charges_sp_0,monthly_charges_sp_1,monthly_charges_sp_2,monthly_charges_sp_3,monthly_charges_sp_4,total_charges_sp_0,total_charges_sp_1,total_charges_sp_2,total_charges_sp_3,total_charges_sp_4,monthly_charges_q_100,total_charges_q_100
0,7020,2020-01-01,NaN,Month-to-month,Yes,Electronic check,29.85,29.85,DSL,No,Yes,No,No,No,No,Female,0,Yes,No,No,0,0.075838,0.619528,0.302584,0.002050,0.000000,0.165395,0.666660,0.167945,2.764200e-09,0.0,0.232137,0.024832
1,7021,2017-04-01,NaN,One year,No,Mailed check,56.95,1889.50,DSL,Yes,No,Yes,No,No,No,Male,0,No,No,No,0,0.002024,0.301936,0.619907,0.076133,0.000000,0.030585,0.520509,0.435494,1.341205e-02,0.0,0.389899,0.578927
2,7022,2019-10-01,2019-12-01,Month-to-month,Yes,Mailed check,53.85,108.15,DSL,Yes,Yes,No,No,No,No,Male,0,No,No,No,1,0.004130,0.342546,0.594060,0.059264,0.000000,0.156567,0.666246,0.177185,1.461391e-06,0.0,0.353535,0.116497
3,7023,2016-05-01,NaN,One year,No,Bank transfer (automatic),42.30,1840.75,DSL,Yes,No,Yes,Yes,No,No,Male,0,No,No,No,0,0.023624,0.492418,0.465686,0.018272,0.000000,0.032438,0.527033,0.428139,1.239059e-02,0.0,0.267536,0.571996
4,7024,2019-09-01,2019-11-01,Month-to-month,Yes,Electronic check,70.70,151.65,Fiber optic,No,No,No,No,No,No,Female,0,No,No,No,1,0.000000,0.145721,0.664792,0.189474,0.000014,0.151802,0.665741,0.182452,4.803606e-06,0.0,0.505612,0.138627
